In [1]:
import seaborn as sns
sns.set()

In [306]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import datetime as dt

# Time Series Data: Predict Temperature

Time series prediction presents its own challenges which are different from machine-learning problems.  As with many other classes of problems, there are a number of common features in these predictions.


## A note on scoring

It **is** possible to score >1 on these questions. This indicates that you've beaten our reference model - we compare our model's score on a test set to your score on a test set. See how high you can go!


## Fetch the data:

The data can be loaded into pandas easily:

In [386]:
df = pd.read_csv('train.v2.csv.gz')
df

,station,time,temp,dew_point,pressure,wind_speed,wind_direction,precip_hour,weather_codes
0,PHX,2010-01-01 00:51,62.06,15.98,1024.90,3.00,20.00,M,M
1,PHX,2010-01-01 01:51,60.08,17.96,1025.30,4.00,50.00,M,M
2,PHX,2010-01-01 02:51,59.00,17.96,1025.60,4.00,30.00,M,M
3,PHX,2010-01-01 03:51,53.96,21.92,1026.00,0.00,0.00,M,M
4,PHX,2010-01-01 04:51,55.94,17.06,1026.20,5.00,40.00,M,M
...,...,...,...,...,...,...,...,...,...
392131,NYC,2018-12-31 19:51,43.00,37.90,1024.10,M,M,0.03,-RA
392132,NYC,2018-12-31 20:51,43.00,37.90,1023.40,M,M,0.02,-RA
392133,NYC,2018-12-31 21:51,43.00,39.90,1021.60,M,M,0.04,RA BR
392134,NYC,2018-12-31 22:51,43.00,41.00,1020.90,M,M,0.05,RA BR


The `station` column indicates the city.  The `time` is measured in UTC.  Both `temp` and `dew_point` are measured in degrees Fahrenheit.  The `wind_speed` is in knots, and the `precip_hour` measures the hourly precipitation in inches.

Missing values are indicated by a flag value.  Remove rows without valid temperature measurements.  You may also want to change some data types. 

In [309]:
df.columns

Index(['station', 'time', 'temp', 'dew_point', 'pressure', 'wind_speed',
       'wind_direction', 'precip_hour', 'weather_codes'],
      dtype='object')

In [387]:
for x in df.columns:
    print(x, len(df[df[x]=='M']))

station 0
time 0
temp 116
dew_point 142
pressure 4533
wind_speed 7330
wind_direction 42054
precip_hour 86471
weather_codes 360028


In [311]:
# df['time'] = pd.to_datetime(df['time'])
# # df["a"] = pd.to_numeric(df["a"])

In [388]:
df= df[df['temp']!='M']

In [313]:
df["temp"]=  pd.to_numeric(df["temp"])

In [314]:
for x in df.columns:
    print(x, len(df[df[x]=='M']))

station 0
time 0
temp 0
dew_point 27
pressure 4442
wind_speed 7299
wind_direction 42004
precip_hour 86452
weather_codes 359933


We will focus on using the temporal elements to predict the temperature.


# Questions


For each question, build a model to predict the temperature in a given city at a given time.  You will be given a DataFrame, as we got from `pd.read_csv`.  (As you can imagine, the temperature values will be nonsensical in the DataFrame you are given.)  Return a collection of predicted temperatures, one for each incoming row in the DataFrame.  

In [315]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392020 entries, 0 to 392135
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   station         392020 non-null  object 
 1   time            392020 non-null  object 
 2   temp            392020 non-null  float64
 3   dew_point       392020 non-null  object 
 4   pressure        392020 non-null  object 
 5   wind_speed      392020 non-null  object 
 6   wind_direction  392020 non-null  object 
 7   precip_hour     392020 non-null  object 
 8   weather_codes   392020 non-null  object 
dtypes: float64(1), object(8)
memory usage: 29.9+ MB


## One-city model

As you may have noticed, the data contains rows for multiple cities.  We'll deal with all of them soon, but for this first question, we'll focus on only the data from New York (`"NYC"`).  Start by isolating only those rows.

In [316]:
df_nyc = df[df['station']=='NYC']

In [317]:
for x in df_nyc.columns:
    print(x, len(df[df[x]=='M']))

station 0
time 0
temp 0
dew_point 27
pressure 4442
wind_speed 7299
wind_direction 42004
precip_hour 86452
weather_codes 359933


Seasonal features are nice because they are relatively safe to extrapolate into the future. There are two ways to handle seasonality.  

The simplest (and perhaps most robust) is to have a set of indicator variables. That is, make the assumption that the temperature at any given time is a function of only the month of the year and the hour of the day, and use that to predict the temperature value.

**Question**: Should month be a continuous or categorical variable?  (Recall that [one-hot encoding](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) is useful to deal with categorical variables.)

Build a model to predict the temperature for a given hour in a given month in New York.

In [318]:
temp = pd.to_numeric(df_nyc['temp'])

In [319]:
from sklearn.base import TransformerMixin, BaseEstimator

class ConvertDateTemp(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        X.loc[:,'time'] = pd.to_datetime(X['time'])
#         if 'temp' in X.columns:
#             X.loc[:,'temp'] =  pd.to_numeric(X["temp"])
        return  X
    
convert_date_temp = ConvertDateTemp()

In [320]:
class MonthHourExtractor(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X will be a pandas series. Return a pandas series of dictionaries
        return pd.DataFrame([[i.month,i.hour] for i in X.squeeze()])
    
    
from sklearn.compose import ColumnTransformer

selector = ColumnTransformer([
    ('time_temp', MonthHourExtractor(), ['time'])
])

In [321]:
# df_nyc[['time','temp']].set_index('time').plot();

In [322]:
range(2,15,1)

range(2, 15)

In [323]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


r_f_regressor = RandomForestRegressor()
 
# max_depth_range = range(3,100,5)
# min_samples_split_range= range(200,1000,50)
# parameters = {'max_depth': max_depth_range,'min_samples_split' : min_samples_split_range}
# dt = DecisionTreeRegressor()
# dt_grid = GridSearchCV(dt, parameters, verbose=3)

# one_hot = OneHotEncoder()

# n_alphas = 50
# alphas = np.logspace(-10, 3, n_alphas)

# parameters = {'alpha': alphas}
# ridge_ = Ridge()
# ridge_grid = GridSearchCV(ridge_, parameters, verbose=3)


predict_temp = Pipeline([
        ('convert_date_temp',convert_date_temp),
        ('selector',selector),
#         ('one_hot', one_hot),## not needed
#         ('dt_grid',dt_grid) 
        ('ran_forest_regressot', RandomForestRegressor())## RandomForestRegressor will also yeild same results
], verbose=True)

In [324]:
predict_temp.fit(df_nyc, temp)

[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.5s
[Pipeline]  (step 3 of 3) Processing ran_forest_regressot, total=   1.6s


Pipeline(steps=[('convert_date_temp', ConvertDateTemp()),
                ('selector',
                 ColumnTransformer(transformers=[('time_temp',
                                                  MonthHourExtractor(),
                                                  ['time'])])),
                ('ran_forest_regressot', RandomForestRegressor())],
         verbose=True)

In [325]:
# predict_temp['dt_grid'].best_estimator_

In [326]:
pd.Series(predict_temp.predict(df_nyc))

0        33.910696
1        33.411322
2        32.923107
3        32.567538
4        32.133810
           ...    
77740    42.978126
77741    42.551293
77742    41.787184
77743    41.462448
77744    41.071648
Length: 77745, dtype: float64

## Per-city model

Now we want to extend this same model to handle all of the cities in our data set.  Rather than adding features to the existing model to handle this, we'll just make a new copy of the model for each city.

If your model is a single class, then this is easy&mdash;you can just instantiate your class once per city.  But it's more likely your model was a particular instance of a Pipeline.  If that's the case, make a **factory function** that returns a new copy of that Pipeline each time it's called.

In [328]:
from sklearn.ensemble import RandomForestRegressor
def season_factory():
    return Pipeline([
        ('convert_date_temp',convert_date_temp),
        ('selector',selector),
#         ('one_hot', one_hot),## not needed
#         ('dt_grid',dt_grid) 
        ('ran_forest_regressot', RandomForestRegressor())## RandomForestRegressor will also yeild same results
], verbose=True) # A single estimator or a pipeline

Calling this function should give a new copy of the Pipeline.  If we train that new copy on the New York data, it should give us the same model as before. 

While we could manually call this function for each city in our dataset, let's build a "group-by" estimator that does this for us.  This estimator should take a column name and a factory function as an argument.  The `fit` method will group the incoming data by that column, and for each group it will call the factory to create a new instance to be trained by on that group.  Then, the `predict` method should look up the corresponding model for each row and perform a predict using that model.

In [329]:
# from sklearn import base
# import pandas as pd 
# import datetime
# import numpy as np
# class GroupbyEstimator(base.BaseEstimator, base.RegressorMixin):
    
#     def __init__(self, column, estimator_factory):
#         self.column = column
#         self.estimator_factory = estimator_factory
#         self.estimators ={}
#         # column is the value to group by; estimator_factory can be
#         # called to produce estimators
    
#     def fit(self, X, y):
#         X['time'] = pd.to_datetime(X['time'])
#         X['month'] = X['time'].dt.month
#         X['hour'] = X['time'].dt.hour
#         y =  pd.to_numeric(y)
        
#         groups = X[self.column].unique()
#         for i in groups:
#             slicer = X[self.column] == i
#             X_i = X[slicer][['month','hour']]
#             y_i = y[X[self.column] == i]
#             self.estimators[i] = self.estimator_factory().fit(X_i,y_i)
#         # Create an estimator and fit it with the portion in each group
#         return self

#     def predict(self, X):
#         X['time'] = pd.to_datetime(X['time'])
#         X['month'] = X['time'].dt.month
#         X['hour'] = X['time'].dt.hour
#         # Call the appropriate predict method for each row of X
        
#         predictions = X.apply(lambda row : self.estimators[row[self.column]].predict(np.array([row['month'],row['hour']]).reshape(1, -1)), axis = 1)
# #         predictions = []
# #         for i, row in X.iterrows():
# #             identifier = row[self.column]
# #             predictions.append(self.estimators[identifier].predict([[row['month'],row['hour']]]))
#         return [int(i) for i in predictions]

In [330]:
from sklearn import base
import pandas as pd 
import datetime
import numpy as np
class GroupbyEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self, column, estimator_factory):
        self.column = column
        self.estimator_factory = estimator_factory
        self.estimators ={}
        # column is the value to group by; estimator_factory can be
        # called to produce estimators
    
    def fit(self, X, y):
        y =  pd.to_numeric(y)
        
        groups = X[self.column].unique()
        for i in groups:
            slicer = X[self.column] == i
            X_i = X[slicer]
            y_i = y[slicer]
            self.estimators[i] = self.estimator_factory().fit(X_i,y_i)
        # Create an estimator and fit it with the portion in each group
        return self

    def predict(self, X):
        # Call the appropriate predict method for each row of X
        index_ = []
        predictions = []
        groups = X[self.column].unique()
        X = X.reset_index()
        for i in groups:
            slicer = X[self.column] == i
            X_i = X[slicer]
            pred = list(self.estimators[i].predict(X_i))
            index_.extend(list(X_i.index))
            predictions.extend(pred)
        total_prediction = pd.DataFrame({'id': index_, 'pred': predictions }).sort_values('id')

        return np.array(total_prediction['pred'])

Now, we should be able to build an equivalent model for each city:

In [331]:
season_model = GroupbyEstimator('station', season_factory).fit(df, df['temp'])

[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.5s
[Pipeline]  (step 3 of 3) Processing ran_forest_regressot, total=   1.6s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline]  (step 3 of 3) Processing ran_forest_regressot, total=   1.6s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.5s
[Pipeline]  (step 3 of 3) Processing ran_forest_regressot, total=   1.7s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline]  (step 3 of 3) Processing ran_forest_regressot, total=   1.6s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.5

In [332]:
df.loc[ np.r_[ 1,2,3, 392133,392132,392131, 4, 5],:]

,station,time,temp,dew_point,pressure,wind_speed,wind_direction,precip_hour,weather_codes
1,PHX,2010-01-01 01:51,60.08,17.96,1025.30,4.00,50.00,M,M
2,PHX,2010-01-01 02:51,59.00,17.96,1025.60,4.00,30.00,M,M
3,PHX,2010-01-01 03:51,53.96,21.92,1026.00,0.00,0.00,M,M
392133,NYC,2018-12-31 21:51,43.00,39.90,1021.60,M,M,0.04,RA BR
392132,NYC,2018-12-31 20:51,43.00,37.90,1023.40,M,M,0.02,-RA
392131,NYC,2018-12-31 19:51,43.00,37.90,1024.10,M,M,0.03,-RA
4,PHX,2010-01-01 04:51,55.94,17.06,1026.20,5.00,40.00,M,M
5,PHX,2010-01-01 05:51,53.06,17.06,1026.20,7.00,50.00,M,M


Submit the same model again to the following scorer:

If you passed, congratulations&mdash;you avoided a common pitfall!  Move on to the next question.

But if your model suddenly behaved worse: In the previous question, we provided each city's rows in contiguous groups.  In this question, the rows were all shuffled together.  If you were predicting for a group at a time and just appending those grouped predictions for the final output, it'll be in the wrong order.

There are two ways to fix this:
1. Predict for each row individually.  This is straightforward, but very, _very_ slow.
2. Predict for each group, and then reorder the predictions to match the input order.  A common way to do this is to attach the index of the feature matrix to the predictions, and then order the full prediction series by the index of the feature matrix.

Once you've fixed your `GroupbyEstimator.predict` method, resubmit to this question.

## Fourier model

Let's consider another way to deal with the seasonal terms.  Since we know that temperature is roughly sinusoidal, we know that a reasonable model might be

$$ y_t = y_0 \sin\left(2\pi\frac{t - t_0}{T}\right) + \epsilon $$

where $y_0$ and $t_0$ are parameters to be learned and $T$ is the period - one year for seasonal variation, one day for daily, etc.  While this is linear in $y_0$, it is not linear in $t_0$. However, we know from Fourier analysis, that the above is
equivalent to

$$ y_t = A \sin\left(2\pi\frac{t}{T}\right) + B \cos\left(2\pi\frac{t}{T}\right) + \epsilon $$

which is linear in $A$ and $B$.

Create a model containing sinusoidal terms on one or more time scales, and fit it to the data using a linear regression.  Build a `fourier_factory` function that will return instances of this model.

In [365]:
np.array(pd.DatetimeIndex(df['time']).to_julian_date())

array([2455197.53541667, 2455197.57708333, 2455197.61875   , ...,
       2458484.41041667, 2458484.45208333, 2458484.49375   ])

In [378]:
from sklearn.base import TransformerMixin, BaseEstimator

class ConvertDateTemp(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        X['time'] = pd.to_datetime(X['time'])
        X['Julian'] = np.array(pd.DatetimeIndex(X['time']).to_julian_date())
        X['const'] = 1
        X['sin_year'] = np.sin(X['Julian'] / 365.25 * 2 * np.pi)
        X['cos_year'] = np.cos(X['Julian'] / 365.25 * 2 * np.pi)
        X['sin_6mo'] = np.sin(X['Julian'] / (365.25 / 2) * 2 * np.pi)
        X['cos_6mo'] = np.cos(X['Julian'] / (365.25 / 2) * 2 * np.pi)
        X['sin_day'] = np.sin(X['time'].dt.hour / 24.0 * 2* np.pi)
        X['cos_day'] = np.cos(X['time'].dt.hour / 24.0 * 2* np.pi)
        return  X
    
convert_date_temp = ConvertDateTemp()



In [380]:
convert_date_temp.fit_transform(df).columns

Index(['station', 'time', 'temp', 'dew_point', 'pressure', 'wind_speed',
       'wind_direction', 'precip_hour', 'weather_codes', 'Julian', 'const',
       'sin_year', 'cos_year', 'sin_6mo', 'cos_6mo', 'sin_day', 'cos_day'],
      dtype='object')

In [381]:
# class MonthHourExtractor(BaseEstimator, TransformerMixin):
    
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
# #         temps['Julian'] = np.array(pd.DatetimeIndex(df['time']).to_julian_date())
# #         temps['const'] = 1
# #         temps['sin(year)'] = np.sin(temps['Julian'] / 365.25 * 2 * np.pi)
# #         temps['cos(year)'] = np.cos(temps['Julian'] / 365.25 * 2 * np.pi)
# #         temps['sin(6mo)'] = np.sin(temps['Julian'] / (365.25 / 2) * 2 * np.pi)
# #         temps['cos(6mo)'] = np.cos(temps['Julian'] / (365.25 / 2) * 2 * np.pi)
# #         temps['sin(day)'] = np.sin(temps.index.hour / 24.0 * 2* np.pi)
# #         temps['cos(day)'] = np.cos(temps.index.hour / 24.0 * 2* np.pi)
        
        
#         return X.to_julian_date()#pd.DataFrame([[i.month,i.hour] for i in X.squeeze()])
    
    
from sklearn.compose import ColumnTransformer

selector = ColumnTransformer([
    ('time_temp','passthrough', ['sin_year', 'cos_year', 'sin_6mo', 'cos_6mo', 'sin_day', 'cos_day']) 
])

In [382]:
selector.fit_transform(convert_date_temp.fit_transform(df))

array([[-0.22117804,  0.97523345, -0.43140044,  0.90216055,  0.        ,
         1.        ],
       [-0.22047896,  0.97539173, -0.43010671,  0.90277805,  0.25881905,
         0.96592583],
       [-0.21977978,  0.97554951, -0.42881211,  0.9033937 ,  0.5       ,
         0.8660254 ],
       ...,
       [-0.22746453,  0.97378637, -0.44300372,  0.89651977, -0.70710678,
         0.70710678],
       [-0.22676649,  0.97394916, -0.44171807,  0.89715391, -0.5       ,
         0.8660254 ],
       [-0.22606834,  0.97411144, -0.44043152,  0.89778621, -0.25881905,
         0.96592583]])

In [383]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

predict_temp = Pipeline([
        ('convert_date_temp',convert_date_temp),
        ('selector',selector),
#         ('one_hot', one_hot),## not needed
#         ('dt_grid',dt_grid) 
        ('l_regressor', LinearRegression())## RandomForestRegressor will also yeild same results
], verbose=True)

In [384]:
def fourier_factory():
    return Pipeline([
        ('convert_date_temp',convert_date_temp),
        ('selector',selector),
#         ('one_hot', one_hot),## not needed
#         ('dt_grid',dt_grid) 
        ('l_regressor', LinearRegression())## RandomForestRegressor will also yeild same results
], verbose=True)

A general `GroupByEstimator` should be able to take the new factory function and build a model for each city.

In [392]:
fourier_model = GroupbyEstimator('station', fourier_factory).fit(df, df['temp'])

[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.1s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ....... (step 3 of 3) Processing l_regressor, total=   0.0s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ....... (step 3 of 3) Processing l_regressor, total=   0.0s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ....... (step 3 of 3) Processing l_regressor, total=   0.0s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipeline] ....... (step 3 of 3) Processing l_regressor, total=   0.0s
[Pipeline] . (step 1 of 3) Processing convert_date_temp, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.0s
[Pipel

*Copyright &copy; 2022 Pragmatic Institute. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*